print(train_df.shape)
print(test_df.shape)
print(bids_df.shape)

(2013, 4)
(4700, 3)
(7656334, 8)

In [27]:
import pandas as pd

bids_df = pd.read_csv('data/bids.csv')
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

bidder_unique = bids_df.groupby("bidder_id").nunique().reset_index()
bidder_unique = bidder_unique.rename(columns  = {"auction" : "auction_nunique" , "merchandise" : "merchandise_nunique" ,
                                              "device" : "device_nunique" ,"time" : "time_nunique" ,
                                               "country" : "country_nunique" ,"ip" : "ip_nunique" ,
                                               "url" : "url_nunique"                                               
                                              })

bidder_unique["country"] = bids_df["country"]
train_set = train_df.merge(bidder_unique, how='left')
test_set = test_df.merge(bidder_unique, how='left')



In [28]:
# train_set = train_df.merge(bids_df, on='bidder_id', how='left')

print(train_set.shape) 
print(test_set.shape) 
print(bidder_unique.shape)



(2013, 12)
(4700, 11)
(6614, 9)


In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


def test_model(feature_cols,train_set):

    X = train_set[feature_cols]
    y = train_set['outcome']
    
    ##Random Forest Classifier
    rf_model = RandomForestClassifier(random_state = 0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    rf_model = rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)

    print('Accuracy:', metrics.accuracy_score(y_test, y_pred) * 100, '%')
    print('AUC:', metrics.roc_auc_score(y_test, y_pred)) # performs much worse that the leaderboard :p
    
def concad(train_set,column):
    print("concad shape part1", train_set.shape)
    
    train_set2 = pd.get_dummies(train_set[column])
    train_set3 = pd.concat([train_set, train_set2], axis=1)
    
    print("concad shape part2", train_set.shape)
    
    train_set3 = train_set3.drop(columns = [column]) 
    return train_set3

def transform(train_set):
    train_set = concad(train_set,"country")
    
#     train_set = train_set.merge(bidder_unique, on='bidder_id', how='left')
    
    print("train mode", train_set["device_nunique"].mode())
    
#     train_set = train_set.drop(columns = ["merchandise", "time" , "ip" , "url" , "payment_account" , "address" , "payment_account" , "device", "auction" , "merchandise_nunique"])
    train_set = train_set.drop(columns = ["address" , "payment_account"])
    train_set = train_set.fillna(train_set.mode())
    train_set["device_nunique"] = train_set["device_nunique"].fillna(train_set["device_nunique"].mode()[0])
    train_set["time_nunique"] = train_set["time_nunique"].fillna(train_set["time_nunique"].mode()[0])
    train_set["country_nunique"] = train_set["country_nunique"].fillna(train_set["country_nunique"].mode()[0])
    train_set["ip_nunique"] = train_set["ip_nunique"].fillna(train_set["ip_nunique"].mode()[0])
    train_set["url_nunique"] = train_set["url_nunique"].fillna(train_set["url_nunique"].mode()[0])
    train_set["auction_nunique"] = train_set["auction_nunique"].fillna(train_set["auction_nunique"].mode()[0])
    train_set["merchandise_nunique"] = train_set["merchandise_nunique"].fillna(train_set["merchandise_nunique"].mode()[0])
    return train_set

In [66]:

train_set = transform(train_set)


concad shape part1 (2013, 12)
concad shape part2 (2013, 12)
train mode 0    1.0
dtype: float64


In [67]:
train_set.isnull().sum()

bidder_id              0
outcome                0
auction_nunique        0
merchandise_nunique    0
device_nunique         0
                      ..
ve                     0
vn                     0
za                     0
zm                     0
zw                     0
Length: 119, dtype: int64

In [68]:
feature_cols = list(train_set.columns)
feature_cols.remove("outcome")
feature_cols.remove("bidder_id")
test_model(feature_cols,train_set)

Accuracy: 95.36423841059603 %
AUC: 0.5336131934032983


In [69]:
def output(feature_cols,test_set,train_set):
    X = train_set[feature_cols]
    y = train_set['outcome']

    
    final_rf_model = RandomForestClassifier(random_state = 0)
    final_rf_model = final_rf_model.fit(X, y)
#     feature_cols = list(test_set.columns)
    X_kaggle = test_set[feature_cols]
    probabilities = final_rf_model.predict_proba(X_kaggle)
    
    kaggle_preds = probabilities[:,1]  # extract values from the rightmost column
    output_dataframe = pd.DataFrame({'Id': test_set['bidder_id'],'Predicted': kaggle_preds})
    output_dataframe.drop_duplicates()
    output_dataframe.to_csv('my_predictions.csv', index=False)  

In [70]:
test_set = transform(test_set)

test_set

concad shape part1 (4700, 11)
concad shape part2 (4700, 11)
train mode 0    1.0
dtype: float64


,bidder_id,auction_nunique,merchandise_nunique,device_nunique,time_nunique,country_nunique,ip_nunique,url_nunique,ae,af,...,ug,uk,us,uy,uz,ve,vn,za,zm,zw
0,0013f9b8d1f462df4462e1c1216e441ba6e18,6.0,1.0,7.0,8.0,2.0,8.0,4.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7.0,1.0,7.0,7.0,4.0,7.0,4.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,002828f800c5132e297cfe3d44fbde9aeac51,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0040cea6b93afd86768c365d89513ffb7c0ba,23.0,1.0,21.0,31.0,6.0,28.0,4.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00512db4ae953baed983a4bcfa335e7412013,345.0,1.0,331.0,1304.0,70.0,888.0,616.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4695,ffd49be672b1ba493b07dccf29311045c5392,20.0,1.0,69.0,168.0,5.0,102.0,40.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4696,ffdd8ed91a683b0f2a8237798ebe9214e3c43,4.0,1.0,3.0,5.0,1.0,4.0,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4697,ffde8717e9a547d536a4e9c3f44782829c021,16.0,1.0,20.0,24.0,6.0,24.0,4.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4698,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [79]:
feature_cols = list(train_set.columns)
feature_cols.remove("outcome")
feature_cols.remove("bidder_id")

output(feature_cols,test_set,train_set)

In [76]:

test_extra_col = list(test_set.columns)
train_extra_col = list(train_set.columns)
train_extra_col.remove("outcome")

c = set(test_extra_col) & set(train_extra_col)

for i in c:
    train_extra_col.remove(i)
    test_extra_col.remove(i)

In [40]:
feature_cols = list(train_set.columns)
feature_cols

['bidder_id',
 'outcome',
 'ad',
 'ae',
 'af',
 'ag',
 'al',
 'am',
 'an',
 'ao',
 'ar',
 'at',
 'au',
 'aw',
 'az',
 'ba',
 'bb',
 'bd',
 'be',
 'bf',
 'bg',
 'bh',
 'bi',
 'bj',
 'bm',
 'bn',
 'bo',
 'br',
 'bs',
 'bt',
 'bw',
 'by',
 'bz',
 'ca',
 'cd',
 'cf',
 'cg',
 'ch',
 'ci',
 'cl',
 'cm',
 'cn',
 'co',
 'cr',
 'cv',
 'cy',
 'cz',
 'de',
 'dj',
 'dk',
 'dm',
 'do',
 'dz',
 'ec',
 'ee',
 'eg',
 'er',
 'es',
 'et',
 'eu',
 'fi',
 'fj',
 'fo',
 'fr',
 'ga',
 'gb',
 'ge',
 'gh',
 'gi',
 'gl',
 'gm',
 'gn',
 'gp',
 'gq',
 'gr',
 'gt',
 'gu',
 'gy',
 'hk',
 'hn',
 'hr',
 'ht',
 'hu',
 'id',
 'ie',
 'il',
 'in',
 'iq',
 'ir',
 'is',
 'it',
 'je',
 'jm',
 'jo',
 'jp',
 'ke',
 'kg',
 'kh',
 'kr',
 'kw',
 'kz',
 'la',
 'lb',
 'li',
 'lk',
 'lr',
 'ls',
 'lt',
 'lu',
 'lv',
 'ly',
 'ma',
 'mc',
 'md',
 'me',
 'mg',
 'mh',
 'mk',
 'ml',
 'mm',
 'mn',
 'mo',
 'mp',
 'mr',
 'mt',
 'mu',
 'mv',
 'mw',
 'mx',
 'my',
 'mz',
 'na',
 'nc',
 'ne',
 'ng',
 'ni',
 'nl',
 'no',
 'np',
 'nz',
 'om',
 

In [51]:
feature_cols = list(train_set.columns)
feature_cols.remove("outcome")
print(feature_cols)

['bidder_id', 'ad', 'ae', 'af', 'ag', 'al', 'am', 'an', 'ao', 'ar', 'at', 'au', 'aw', 'az', 'ba', 'bb', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj', 'bm', 'bn', 'bo', 'br', 'bs', 'bt', 'bw', 'by', 'bz', 'ca', 'cd', 'cf', 'cg', 'ch', 'ci', 'cl', 'cm', 'cn', 'co', 'cr', 'cv', 'cy', 'cz', 'de', 'dj', 'dk', 'dm', 'do', 'dz', 'ec', 'ee', 'eg', 'er', 'es', 'et', 'eu', 'fi', 'fj', 'fo', 'fr', 'ga', 'gb', 'ge', 'gh', 'gi', 'gl', 'gm', 'gn', 'gp', 'gq', 'gr', 'gt', 'gu', 'gy', 'hk', 'hn', 'hr', 'ht', 'hu', 'id', 'ie', 'il', 'in', 'iq', 'ir', 'is', 'it', 'je', 'jm', 'jo', 'jp', 'ke', 'kg', 'kh', 'kr', 'kw', 'kz', 'la', 'lb', 'li', 'lk', 'lr', 'ls', 'lt', 'lu', 'lv', 'ly', 'ma', 'mc', 'md', 'me', 'mg', 'mh', 'mk', 'ml', 'mm', 'mn', 'mo', 'mp', 'mr', 'mt', 'mu', 'mv', 'mw', 'mx', 'my', 'mz', 'na', 'nc', 'ne', 'ng', 'ni', 'nl', 'no', 'np', 'nz', 'om', 'pa', 'pe', 'pf', 'pg', 'ph', 'pk', 'pl', 'pr', 'ps', 'pt', 'py', 'qa', 're', 'ro', 'rs', 'ru', 'rw', 'sa', 'sb', 'sc', 'sd', 'se', 'sg', 'si', 'sk', 's